In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np

## Load Business Category Data

In [2]:
# Get top-level business categories
with open("categories.json") as json_file:
    categories = json.load(json_file)

# Dictionary of category keys to the full strings that appear in the dataset
category_dict = {category["alias"]:category["title"] for category in categories}
category_dict_reverse = {category["title"]:category["alias"] for category in categories}

# Categories that we are interested in
#categories_of_interest = ['active', 'arts', 'beautysvc', 'food', 'hotelstravel', 'nightlife', 'restaurants', 'shopping']
categories_of_interest_raw = [category["alias"] for category in categories if category["parents"] == []]
categories_of_interest = [category_dict[cat] for cat in categories_of_interest_raw]

# print(categories_of_interest)
# print(categories_of_interest_raw)

## Business Data

### Load Dataset

In [3]:
businesses = pd.read_csv("YelpChallengeWMetros_Clean.csv")

# businesses.head(3)

### Process Categories

In [4]:
# Create indicator columns for each business category (a business may belong to multiple categories)
businesses_categories = businesses.copy()

for category_str in categories_of_interest:
    colname = "is_" + category_dict_reverse[category_str]
    businesses_categories[colname] = businesses_categories["categories"].str.contains(category_str).fillna(False)
# businesses_categories.head(3)

## Counts Data (all metro areas combined) and num_businesses

### counts_df

In [5]:
metros = ['Boston', 'Portland', 'Austin', 'Orlando', 'Atlanta', 'Vancouver', 'Columbus', 'Boulder']
v2 = True

counts_df = pd.DataFrame()
# counts_df_norm = pd.DataFrame()

for metro in metros:
    if v2:
        suffix = "_v2.csv"
    else:
        suffix = ".csv"
        
    #counts_df_metro = pd.read_csv("output/" + metro + "_counts_final" + suffix)
    counts_df_metro = pd.read_csv("output_urbcomp/" + metro + "_counts_final" + suffix)
    counts_df_metro["metroarea"] = metro
    counts_df = counts_df.append(counts_df_metro)
    
#     counts_df_norm_metro = pd.read_csv("output/" + metro + "_counts_norm_final" + suffix)
#     counts_df_norm_metro["metroarea"] = metro
#     counts_df_norm = counts_df_norm.append(counts_df_norm_metro)

# counts_df.head()
#counts_df.to_csv("test/all_counts_final_v2.csv", index=False)
counts_df.to_csv("final_output_urbcomp/all_counts_final_v2.csv", index=False)
# counts_df_norm.to_csv("test/all_counts_norm_final_v2.csv", index=False)

#### num_businesses

In [6]:
categories_of_interest_small = ['active', 'arts', 'beautysvc', 'food', 'hotelstravel', 'nightlife', 'restaurants', 'shopping']
counts_df_mod_noprofs = counts_df.copy()
for col in ["is_"+col for col in categories_of_interest_small if col != "restaurants"]:
    counts_df_mod_noprofs[col] = np.logical_and(counts_df_mod_noprofs[col] == True, counts_df_mod_noprofs["is_restaurants"] == False)
cols = ["is_"+col for col in categories_of_interest_small]
#len(counts_df_mod[counts_df_mod[cols].sum(axis=1) > 1]) / len(counts_df_mod)

df_rows = []
# categories_of_interest_small = ['active', 'arts', 'beautysvc', 'food', 'hotelstravel', 'nightlife', 'restaurants', 'shopping']

cols = [counts_df.columns[0]] + list(counts_df.columns[3:45])
# print(cols)

for category in categories_of_interest_small:
    df_row = [category]
    q = "is_" + category
    counts_category = counts_df_mod_noprofs.query(q)
    num_bus_in_cat = len(counts_category)
    counts_df_temp = counts_category[cols[1:]]
    #print(np.count_nonzero(counts_df_temp))
    df_row.append(num_bus_in_cat)
    df_row.extend(np.count_nonzero(counts_df_temp, axis=0))
    df_rows.append(df_row)
    
num_businesses_df = pd.DataFrame(df_rows,columns = ["category", "num_businesses"] + cols[1:])
    
# num_businesses_df

#num_businesses_df.to_csv("test/num_businesses.csv", index=False)
num_businesses_df.to_csv("final_output_urbcomp/num_businesses.csv", index=False)

### counts_df_norm and table

#### Initial Data Prep

In [7]:
metros = ['Boston', 'Portland', 'Austin', 'Orlando', 'Atlanta', 'Vancouver', 'Columbus', 'Boulder']
v2 = True

# counts_df = pd.DataFrame()
counts_df_norm = pd.DataFrame()

for metro in metros:
    if v2:
        suffix = "_v2.csv"
    else:
        suffix = ".csv"
        
#     counts_df_metro = pd.read_csv("output/" + metro + "_counts_final" + suffix)
#     counts_df_metro["metroarea"] = metro
#     counts_df = counts_df.append(counts_df_metro)
    
    #counts_df_norm_metro = pd.read_csv("output/" + metro + "_counts_norm_final" + suffix)
    counts_df_norm_metro = pd.read_csv("output_urbcomp/" + metro + "_counts_norm_final" + suffix)
    counts_df_norm_metro["metroarea"] = metro
    counts_df_norm = counts_df_norm.append(counts_df_norm_metro)

# counts_df.head()
# counts_df.to_csv("test/all_counts_final_v2.csv", index=False)
# counts_df_norm.to_csv("test/all_counts_norm_final_v2.csv", index=False)

#### Table

In [8]:
metros = ['Boston', 'Portland', 'Austin', 'Orlando', 'Atlanta', 'Vancouver', 'Columbus', 'Boulder']
metros = sorted(metros)#, reverse=True)
#metros.append("Total")
#metros.reverse()

# print(metros)

In [9]:
cols = list(counts_df_norm.columns[3:41])
# cols.remove("professor")
# len(cols)#.remove('professor')
# print(cols)

In [10]:
# counts_df_norm = pd.read_csv("output/" + "all" + "_counts_norm_final_v2.csv")
df = pd.DataFrame(counts_df_norm[cols].mean().sort_values(ascending=False))
#df

for metro in metros:
    #counts_df_norm_metro = pd.read_csv("output/" + metro + "_counts_norm_final_v2.csv")
    counts_df_norm_metro = pd.read_csv("output_urbcomp/" + metro + "_counts_norm_final_v2.csv")
    df[metro] = pd.DataFrame(counts_df_norm_metro[cols].mean().sort_values(ascending=False))
df = df.reset_index()
#df
df = df.rename(columns={"index": "Word", 0: "Total"})
df

,Word,Total,Atlanta,Austin,Boston,Boulder,Columbus,Orlando,Portland,Vancouver
0,friend,33.804432,38.170363,29.798229,36.803044,27.865674,31.683689,25.935353,29.254275,50.024824
1,husband,25.995269,23.441352,27.714613,23.072557,26.335683,33.492668,33.658416,26.555384,17.086610
2,wife,17.656021,14.339346,17.078982,16.194896,18.335786,25.496619,24.312638,17.524192,12.630865
3,boyfriend,12.507301,11.999524,10.701376,15.046056,11.183009,11.239301,12.572060,10.701562,14.608817
4,daughter,9.613763,7.728754,11.358993,8.290963,10.209837,9.889977,13.699108,10.482235,4.655345
5,mother,8.470461,8.058534,7.604727,9.011856,6.987323,9.355530,9.589007,7.967144,8.376795
6,son,8.219784,6.535826,10.509375,6.730260,8.385726,9.432860,11.305546,8.774743,3.941412
7,child,6.745751,5.020901,8.833988,5.342004,6.062716,7.035597,9.361824,7.461666,3.736820
8,girlfriend,5.295986,4.733483,4.647845,6.375159,4.969555,5.282803,5.016992,4.508225,6.515583
9,sister,4.442946,4.678841,3.761399,4.872205,3.512286,4.987914,4.591440,4.100170,4.581450


In [11]:
df.to_latex(buf="output_urbcomp/table.txt", float_format="%.2f", index=False)
#df.to_latex(buf="output/table.txt", float_format="%.2f", index=False)

#### counts_df_norm file

In [12]:
categories_of_interest = ['active', 'arts', 'beautysvc', 'food', 'hotelstravel', 'nightlife', 'restaurants', 'shopping']
q = " or ".join(["is_"+cat for cat in categories_of_interest])
# "(" + q + ") and is_health == False and is_homeservices == False"

In [13]:
final_table = counts_df_norm.query(q)

for col in ["is_"+col for col in categories_of_interest if col != "restaurants"]:
    final_table[col] = np.logical_and(final_table[col] == True, final_table["is_restaurants"] == False)

<ipython-input-13-10c4c5e64074>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table[col] = np.logical_and(final_table[col] == True, final_table["is_restaurants"] == False)
<ipython-input-13-10c4c5e64074>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table[col] = np.logical_and(final_table[col] == True, final_table["is_restaurants"] == False)
<ipython-input-13-10c4c5e64074>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [14]:
#final_table.to_csv("test/all_counts_norm_final_v2.csv", index=False)
final_table.to_csv("final_output_urbcomp/all_counts_norm_final_v2.csv", index=False)

#### counts_df_norm_formaps file

In [15]:
#counts_df_norm.head()

cols_of_interest = ['business_id', 'num_reviews', 'num_relationship_words',
                    'family', 'romantic', 'friendship', 'professional',
                    'name', 'categories', 'stars', 'review_count',
                    'address', 'city', 'state', 'longitude', 'latitude', 'metroarea']

cols_of_interest.extend(["is_"+cat for cat in categories_of_interest])
# print(cols_of_interest)

In [16]:
counts_df_norm_fewcols = final_table[cols_of_interest]
# businesses_renamed = businesses.rename(columns={"business": "business_id"})
# counts_df_norm_merged = pd.merge(counts_df_norm_fewcols, businesses_renamed, how="inner", on="business_id")
#counts_df_norm_fewcols.to_csv("test/all_counts_norm_fewcols_formaps.csv", index=False)
counts_df_norm_fewcols.to_csv("final_output_urbcomp/all_counts_norm_fewcols_formaps.csv", index=False)